In [7]:
# 파이토치 

import torch
import torch.nn as nn

class Sepconv(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(Sepconv, self).__init__()
        layers = []
        layers += [nn.Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=kernel_size, 
                             stride=stride, padding=padding, groups=in_channels, bias=False),
                   nn.BatchNorm2d(in_channels),
                   nn.ReLU(True),
                   nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1),
                   nn.BatchNorm2d(out_channels),
                   nn.ReLU(True)]
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

class MobileNet(nn.Module):

    def __init__(self, in_channels=3, out_channels=1000):
        super(MobileNet, self).__init__()
        layers = []
        layers += [nn.Conv2d(in_channels=in_channels, out_channels=32, kernel_size=3, stride=2, padding=1, bias=False),
                   nn.BatchNorm2d(32),
                   nn.ReLU(True),
                   Sepconv(32, 64),
                   Sepconv(64, 128, stride=2),
                   Sepconv(128, 128),
                   Sepconv(128, 256, stride=2),
                   Sepconv(256, 256),
                   Sepconv(256, 512, stride=2)]
        for i in range(5):
            layers += [Sepconv(512, 512)]
        layers += [Sepconv(512, 1024, stride=2),
                   Sepconv(1024, 1024, stride=2),
                   nn.AdaptiveAvgPool2d((1,1)),
                   View(-1),
                   nn.Linear(1024, out_channels)]

        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

class View(nn.Module):

    def __init__(self, *shape):
        super(View, self).__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(x.shape[0], *self.shape)

if __name__ == '__main__':
    from torchsummary import summary
    model = MobileNet()
    summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              ReLU-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
              ReLU-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 64, 112, 112]           2,112
       BatchNorm2d-8         [-1, 64, 112, 112]             128
              ReLU-9         [-1, 64, 112, 112]               0
          Sepconv-10         [-1, 64, 112, 112]               0
           Conv2d-11           [-1, 64, 56, 56]             576
      BatchNorm2d-12           [-1, 64, 56, 56]             128
             ReLU-13           [-1, 64, 56, 56]               0
           Conv2d-14          [-1, 128,

In [12]:
# 케라스

from keras.models import Model
from keras.layers import Input, SeparableConv2D, Conv2D, BatchNormalization, Activation, Dense, AveragePooling2D, Flatten

def Depthwise_Separable_Conv(pre_layer, dw_num_ch, s):
    x = SeparableConv2D(dw_num_ch, 3, strides=s, padding='same', use_bias=False)(pre_layer)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x

def MobileNet():
    inputs = Input(shape=(224, 224, 3))
    
    x = Conv2D(32, kernel_size=3, strides=2, padding='same', use_bias=False)(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Depthwise_Separable_Conv(x, 64, 1) # 두 번째는 output 채널 수
    x = Depthwise_Separable_Conv(x, 128, 2)
    x = Depthwise_Separable_Conv(x, 128,1)
    x = Depthwise_Separable_Conv(x, 256, 2)
    x = Depthwise_Separable_Conv(x, 256, 1)
    x = Depthwise_Separable_Conv(x, 512, 2)
    
    for num in range(0, 5):
        x = Depthwise_Separable_Conv(x, 512, 1)
        
    x = Depthwise_Separable_Conv(x, 1024, 2)
    x = Depthwise_Separable_Conv(x, 1024, 2)
    x = AveragePooling2D(pool_size=1)(x)
    x = Flatten()(x)
    outputs = Dense(1000, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    
    return model

model = MobileNet()
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 112, 112, 32)      864       
_________________________________________________________________
batch_normalization_85 (Batc (None, 112, 112, 32)      128       
_________________________________________________________________
activation_85 (Activation)   (None, 112, 112, 32)      0         
_________________________________________________________________
separable_conv2d_79 (Separab (None, 112, 112, 64)      2336      
_________________________________________________________________
batch_normalization_86 (Batc (None, 112, 112, 64)      256       
_________________________________________________________________
activation_86 (Activation)   (None, 112, 112, 64)      0   